In [1]:
# import sys
# sys.path.append('/Applications/anaconda3/lib/python3.8/site-packages')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import os
import spindry as spd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdMolAlign
from rdkit import RDLogger

%matplotlib inline

def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()



def rdkit_op(bb):
    rdkit_bb = bb.to_rdkit_mol()
    rdkit.SanitizeMol(rdkit_bb)
    rdkit.MMFFOptimizeMolecule(rdkit_bb)

    # stk molecules are immutable. with_position_matrix returns a
    # a clone, holding the new position matrix.
    bb = bb.with_position_matrix(
        position_matrix=rdkit_bb.GetConformer().GetPositions(),
    )

    return bb

## Creat molecules and xtb optimisation 

In [2]:
metal_name = ['Fe','Co','Ni','Cu','Zn']


# Metal centre porphyrin
metal_centers = [
    stk.BuildingBlock(
        smiles='[Fe+2]',
        functional_groups=(
            stk.SingleAtom(stk.Fe(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),

    stk.BuildingBlock(
        smiles='[Co+2]',
        functional_groups=(
            stk.SingleAtom(stk.Co(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),

    stk.BuildingBlock(
        smiles='[Ni+2]',
        functional_groups=(
            stk.SingleAtom(stk.Ni(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),

    stk.BuildingBlock(
        smiles='[Cu+2]',
        functional_groups=(
            stk.SingleAtom(stk.Cu(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),

    stk.BuildingBlock(
        smiles='[Zn+2]',
        functional_groups=(
            stk.SingleAtom(stk.Zn(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),

]

TPP = stk.BuildingBlock(
    smiles= 'C1(/C2=CC=CC=C2)=C3C=CC(/C(C4=CC=CC=C4)=C5C=C/C([N]/5)=C(C6=CC=CC=C6)/C(C=C/7)=NC7=C(C8=CC=CC=C8)/C9=CC=C1[N]/9)=N/3',
    functional_groups=(
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6!H]~[#7]~[#6!H]',
            bonders=(1,),
            deleters=(),
        ),

    ),
)
TPP = stko.UFF().optimize(TPP)

# Over metal atoms
for ma in range(0,len(metal_centers)):
    M_TPP = stk.ConstructedMolecule(
        topology_graph=stk.metal_complex.Porphyrin(
            metals=metal_centers[ma],
            ligands=TPP,
            optimizer=stk.MCHammer(),
            #optimizer=stk.Collapser(scale_steps=False),
        )
    )

    molecule_name = f'{metal_name[ma]}_TPP'
    print(molecule_name)
                    
  
    # Write to files.
    #porphyrin_noM.write(f'{molecule_name}.mol')
    M_TPP.write(f'{molecule_name}.xyz')
    os.makedirs(f'{molecule_name}')
    os.chdir(f'{molecule_name}')
    os.system(f'mv ../{molecule_name}.xyz .')
    os.environ['XTBHOME'] = "/home/xwu/miniconda3/pkgs/xtb-6.4.1-hf06ca72_0/share/xtb"
    os.system(f'xtb {molecule_name}.xyz --gfn 1 --opt > output_{molecule_name}.txt && xtb xtbopt.xyz --gfn 1 --vipea > vipea_{molecule_name}.txt')
    os.chdir('../') 
    show_stk_mol(M_TPP)
  



Fe_TPP


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Co_TPP


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Ni_TPP


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Cu_TPP


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Zn_TPP


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Abstract HOMO-LUMO Gap & Electron affinity (EA) & Ionisation potential (IP) values after xTB optimisation 

In [1]:
metal_name = ['Fe','Co','Ni','Cu','Zn']

import glob
all_gap_value = []
for metal in metal_name:
    file_location = os.path.join(f'{metal}_TPP',f'output_{metal}_TPP.txt')
    filenames = glob.glob(file_location)
    datafile1 = open('all_HL-gap_MTPP.txt', 'w')
    output = open(file_location,'r')
    data = output.readlines()
    output.close()
    for line in data:
        if 'HOMO-LUMO GAP' in line:
            gap = line
            words = gap.split()
            gap_value =float(words[3])
            gap_value_free = f'H-L gap {metal}_TPP = {gap_value}'
            all_gap_value.append(gap_value_free)
            list = all_gap_value
            datafile1.write("\n".join([i for i in list[0:]]))
    datafile1.close()


all_EA_value = []
all_IP_value = []
for metal in metal_name:
            file_location = os.path.join(f'{metal}_TPP',f'vipea_{metal}_TPP.txt')
            filenames = glob.glob(file_location)
            datafile2 = open('all_EA_MTPP.txt', 'w')
            datafile3 = open('all_IP_MTPP.txt', 'w')
            output = open(file_location,'r')
            data = output.readlines()
            output.close()
            for line in data:
                if 'delta SCC EA (eV):' in line:
                    EA = line
                    words = EA.split()
                    EA_value =float(words[4])
                    EA_value_free = f'EA {metal}_TPP = {EA_value}'
                    all_EA_value.append(EA_value_free)
                    list = all_EA_value
                    datafile2.write("\n".join([i for i in list[0:]]))

                if 'delta SCC IP (eV):' in line:
                    IP = line
                    words = IP.split()
                    IP_value =float(words[4])
                    IP_value_free = f'IP {metal}_TPP = {IP_value}'
                    all_IP_value.append(IP_value_free)
                    list = all_IP_value
                    datafile3.write("\n".join([i for i in list[0:]]))
            datafile2.close()
            datafile3.close()



## Put IP, EA, and H-L gap vaules in a single file 

In [2]:
HLvalue = [round(float(x.split(' ')[4].strip('\n')),3) for x in open('all_HL-gap_MTPP.txt').readlines()]
IPvalue = [round(float(x.split(' ')[3].strip('\n')),3) for x in open('all_IP_MTPP.txt').readlines()]
EAvalue = [round(float(x.split(' ')[3].strip('\n')),3) for x in open('all_EA_MTPP.txt').readlines()]

together = []
for i in range(len(HLvalue)):
    all_values = open('IP_EA_HL_data_MTPP.txt', 'w+')
    all_values.write(f'IP, EA, H-L gap\n')
    values = [IPvalue[i], EAvalue[i], HLvalue[i]]
    together.append(str(values))
    all_values.write("\n".join(together))
all_values.close()

## Get descriptors for each xtb optimised molecule

In [3]:
from rdkit import Chem 
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.ML.Descriptors import MoleculeDescriptors
import numpy as np
import glob

def get_descriptors(rdmols):

#returns the physicochemical properties for the molecules
    
    descriptors = [
        'ExactMolWt', # The exact molecular weight of the molecule
        'NumValenceElectrons', # The number of valence electrons the molecule has
        'HeavyAtomCount', # Number of heavy atoms a molecule.
        'NHOHCount', # Number of NHs or OHs
        'NOCount', # Number of Ns and Os
        'NumHAcceptors', # Number of Hydrogen Bond Acceptors
        'NumHDonors', # of Hydrogen Bond Donors
        'NumHeteroatoms', # Number of Heteroatoms
        'fr_Al_COO', # Number of aliphatic carboxylic acids
        'fr_Al_OH', # Number of aliphatic hydroxyl groups
        'fr_Al_OH_noTert', # Number of aliphatic hydroxyl groups excluding tert-OH
        'fr_COO', # Number of carboxylic acids
        'fr_COO2', # Number of carboxylic acids
        'fr_C_O', # Number of carbonyl O
        'fr_C_O_noCOO', # Number of carbonyl O, excluding COOH
        'fr_NH0', # Number of Tertiary amines
        'fr_NH1', # Number of Secondary amines
        'fr_NH2', # Number of Primary amines
        'fr_allylic_oxid', # Number of allylic oxidation sites excluding steroid dienone
        'fr_dihydropyridine', # Number of dihydropyridines                
        'fr_methoxy', # Number of methoxy groups -OCH3
        'fr_nitrile', # Number of nitriles
        'fr_nitro', # Number of nitro groups
        'fr_nitroso', # Number of nitroso groups, excluding NO2
        'fr_piperdine', # Number of piperdine rings

    ]
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors)
        
    Desc_values = calculator.CalcDescriptors(rdmols)
    all_dec = []
    for i in Desc_values:
        # round each figure to 3 decimal places 
        Desc_values_V2 = round(i,3)
        all_dec.append(Desc_values_V2)
    return all_dec

metal_name = ['Fe','Co','Ni','Cu','Zn']
all_desc_value = []
for metal in metal_name:
    file_location = os.path.join(f'{metal}_TPP','xtbtopo.mol')
    filenames = glob.glob(file_location)
    datafile4 = open('all_desc_MTPP.txt', 'w')
    output = Chem.MolFromMolFile(file_location, sanitize=False, strictParsing=False)
    all_desc_value.append(str(get_descriptors(output)))
    datafile4.write("\n".join(all_desc_value))

## Creat a finial datafile for machine learning 

In [4]:

HLvalue = [round(float(x.split(' ')[4].strip('\n')),3) for x in open('all_HL-gap_MTPP.txt').readlines()]
IPvalue = [round(float(x.split(' ')[3].strip('\n')),3) for x in open('all_IP_MTPP.txt').readlines()]
EAvalue = [round(float(x.split(' ')[3].strip('\n')),3) for x in open('all_EA_MTPP.txt').readlines()]


# creat a list containing all IP, EA and H-L value
together_V2 = []
for i in range(len(HLvalue)):
    values = [IPvalue[i], EAvalue[i], HLvalue[i]]
    together_V2.append(values)


# creat a list containing all descriptors value
import glob
all_desc_value = []
for metal in metal_name:
    file_location = os.path.join(f'{metal}_TPP','xtbtopo.mol')
    filenames = glob.glob(file_location)
    output = Chem.MolFromMolFile(file_location, sanitize=False, strictParsing=False)
    all_desc_value.append(get_descriptors(output))


# In order to add atomic number of each metal center, number of carbon atom in the linker (0) and number of units (1) into descriptors, creat another list.
name_no = []
metal_AN = ['26','27','28','29','30']
for AN in metal_AN:
    label = [int(AN),int(0),int(1)]
    name_no.append(label)


# Combine the 3 lists created above to assemble the final datasetb
total = []
for i in range(len(HLvalue)):
    tot_values = open('total_data_MTPP.txt', 'w+')
    allv = all_desc_value[i] + name_no[i] + together_V2[i]
    total.append(str(allv))
    tot_values.write("\n".join(total))
tot_values.close()



## Creat a finial datafile with labels 

In [5]:
metal_name = ['Fe','Co','Ni','Cu','Zn']

Descvalue = [x.strip('\n') for x in open('total_data_MTPP.txt').readlines()]

total_l = []
for i in range(len(Descvalue)):
    tot_values = open('total_data_MTPP_label.txt', 'w+')
    tot_values.write(f'Name, ExactMolWt, NumVE, HeavyAtom#, NHOH#, NOC#, NumHAcc#, NumHDo#, NumHeteroatoms, fr_Al_COO, fr_Al_OH, fr_Al_OH_noTert, fr_COO, fr_COO2, fr_C_O, fr_C_O_noCOO, fr_NH0, fr_NH1, fr_NH2, fr_allylic_oxid, fr_dihydropyridine, fr_methoxy, fr_nitrile, fr_nitro, fr_nitroso, fr_piperdine, MetalAN, #linker, #unit, IP, EA,  H-L gap\n')
    allv_l = metal_name [i] + '_TPP ' + Descvalue[i]
    total_l.append(allv_l)
    tot_values.write("\n".join(total_l))
tot_values.close()